<a href="https://colab.research.google.com/github/patrickguevara23/Amigo-secreto/blob/main/Projeto_Diabetes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dataset

https://www.kaggle.com/datasets/tigganeha4/diabetes-dataset-2019

Imports

In [1]:
import pandas as pd
import plotly.express as px
from scipy.stats import pearsonr
import seaborn as sns
import statsmodels.api as sm
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_validate
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

Leitura e Tratamento dos Dados

In [2]:
#Leitura do arquivo com Pandas e salvando o conteúdo do dataset em uma variável
df = pd.read_csv('/content/diabetes_dataset__2019.csv')

#Renomeando as colunas para o português
df = df.rename(columns={
    'Age': 'Idade',
    'Gender': 'Sexo',
    'Family_Diabetes': 'Diabetes_familiar',
    'highBP': 'Pressão_alta',
    'PhysicallyActive': 'Ativo_fisicamente',
    'BMI': 'IMC',
    'Smoking': 'Fumante',
    'Alcohol': 'Ingere_alcool',
    'Sleep': 'Horas_sono',
    'SoundSleep': 'Sono_tranquilo',
    'RegularMedicine': 'Medicamentos_regulares',
    'BPLevel': 'Nível_pressão_sanguínea',
    'Pregancies': 'Gradivez',
    'Pdiabetes': 'Pre_diabetes',
    'UriationFreq': 'Frequência_urinária',
    'Diabetic': 'Diabetes',
})

#Checando se existem dados nulos ou inconsistentes
print(df.info())

#Identificando e tratando dados duplicados
print('O conjunto apresenta uma quantidade de',df.duplicated().sum(),'dados duplicados')
#Aplicando um filtro para remover os dados duplicados
filtro_duplicados = df.duplicated()
df[filtro_duplicados]
df.drop_duplicates(inplace = True)
print('O conjunto apresenta uma quantidade de',df.duplicated().sum(),'dados duplicados')

#Identificando e tratando dados nulos
df.loc[(df['Sexo'] == 'Male') & (df['Gradivez'].isna()),'Gradivez'] = 0 #Substituir valores nulos na coluna 'Gradivez' por 0 exclusivamente para linhas onde o sexo é masculino
print('O conjunto apresenta um total de', df.isna().sum().sum(), 'valores ausentes') #Contar o número total de valores ausentes (NaN) em todo o DataFrame df
df[df.isna().any(axis = 1)] #Identificar e selecionar as linhas do DataFrame df que contêm pelo menos um valor ausente (NaN).
colunas_dropar = ['IMC','Gradivez','Pre_diabetes','Diabetes']
df_sem_nulos = df.dropna(subset = colunas_dropar ).copy() #Remover as linhas onde possuem valores ausentes e criar uma cópia do DataFrame
df_sem_nulos.reset_index(drop = True, inplace = True) #Resetando o índice do dataframe
print('O conjunto apresenta uma quantidade de',df_sem_nulos.isna().sum().sum(),'valores ausentes')#Verificando se ainda existem valores ausentes (NaN)

#Checando se todos os valores ausentes, nulos e/ou inconsistentes foram tratados
print(df_sem_nulos.info())







<class 'pandas.core.frame.DataFrame'>
RangeIndex: 952 entries, 0 to 951
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Idade                    952 non-null    object 
 1   Sexo                     952 non-null    object 
 2   Diabetes_familiar        952 non-null    object 
 3   Pressão_alta             952 non-null    object 
 4   Ativo_fisicamente        952 non-null    object 
 5   IMC                      948 non-null    float64
 6   Fumante                  952 non-null    object 
 7   Ingere_alcool            952 non-null    object 
 8   Horas_sono               952 non-null    int64  
 9   Sono_tranquilo           952 non-null    int64  
 10  Medicamentos_regulares   952 non-null    object 
 11  JunkFood                 952 non-null    object 
 12  Stress                   952 non-null    object 
 13  Nível_pressão_sanguínea  952 non-null    object 
 14  Gradivez                 9

Tratando as Variáveis Categóricas

#Tratando as variáveis categóricas

In [3]:
#Verificando os valores únicos de todas as colunas
for col in df_sem_nulos.columns:
    print(f"Coluna: {col}")
    print(df_sem_nulos[col].unique())
    print("-" * 30)

Coluna: Idade
['50-59' '40-49' 'less than 40' '60 or older']
------------------------------
Coluna: Sexo
['Male' 'Female']
------------------------------
Coluna: Diabetes_familiar
['no' 'yes']
------------------------------
Coluna: Pressão_alta
['yes' 'no']
------------------------------
Coluna: Ativo_fisicamente
['one hr or more' 'less than half an hr' 'none' 'more than half an hr']
------------------------------
Coluna: IMC
[39. 28. 24. 23. 27. 21. 20. 26. 22. 15. 34. 30. 29. 18. 32. 31. 36. 38.
 40. 35. 19. 33. 17. 25. 42. 45.]
------------------------------
Coluna: Fumante
['no' 'yes']
------------------------------
Coluna: Ingere_alcool
['no' 'yes']
------------------------------
Coluna: Horas_sono
[ 8  6 10  7 11  9  4  5]
------------------------------
Coluna: Sono_tranquilo
[ 6  8 10  7 11  4  9  5  3  2  1  0]
------------------------------
Coluna: Medicamentos_regulares
['no' 'yes' 'o']
------------------------------
Coluna: JunkFood
['occasionally' 'very often' 'often' 'alwa

In [4]:
#Substituir valores em colunas específicas com base no dicionário 'colunas'.
mapeamento = {
    'no': 0,
    'yes': 1,
    'Male': 0,
    'Female': 1,
    'mensal': 0,
    'o': 0,
    'High': 'high',
    'Low': 'low',
    'Medium': 'medium',
    ' no': 0,
    'normal ': 'normal'
}

colunas = ['Sexo','Diabetes_familiar','Pressão_alta','Fumante','Ingere_alcool','Medicamentos_regulares',
           'Nível_pressão_sanguínea','Pre_diabetes','Diabetes']

df_sem_nulos[colunas] = df_sem_nulos[colunas].replace(mapeamento)

#Criar um DataFrame com as variáveis categóricas de df_sem_nulos transformadas em variáveis dummies e salvando em uma cópia
df_dummies = pd.get_dummies(df_sem_nulos).copy()

#Converter as colunas booleanas(True/False) para 1 ou 0.
boolean_columns = [
    'Idade_40-49',
    'Idade_50-59',
    'Idade_60 or older',
    'Idade_less than 40',
    'Ativo_fisicamente_less than half an hr',
    'Ativo_fisicamente_more than half an hr',
    'Ativo_fisicamente_none',
    'Ativo_fisicamente_one hr or more',
    'JunkFood_always',
    'JunkFood_occasionally',
    'JunkFood_often',
    'JunkFood_very often',
    'Stress_always',
    'Stress_not at all',
    'Stress_sometimes',
    'Stress_very often',
    'Nível_pressão_sanguínea_high',
    'Nível_pressão_sanguínea_low',
    'Nível_pressão_sanguínea_normal',
    'Pre_diabetes_1',
    'Pre_diabetes_0',
    'Frequência_urinária_not much',
    'Frequência_urinária_quite often'
]

df_dummies[boolean_columns] = df_dummies[boolean_columns].astype(int)

#Verificando se restaram dados inconsistentes
print(df_dummies.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 34 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   Sexo                                    299 non-null    int64  
 1   Diabetes_familiar                       299 non-null    int64  
 2   Pressão_alta                            299 non-null    int64  
 3   IMC                                     299 non-null    float64
 4   Fumante                                 299 non-null    int64  
 5   Ingere_alcool                           299 non-null    int64  
 6   Horas_sono                              299 non-null    int64  
 7   Sono_tranquilo                          299 non-null    int64  
 8   Medicamentos_regulares                  299 non-null    int64  
 9   Gradivez                                299 non-null    float64
 10  Diabetes                                299 non-null    int64 

<ipython-input-4-a2a930d554c0>:19: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_sem_nulos[colunas] = df_sem_nulos[colunas].replace(mapeamento)


In [5]:
#Removendo Outliers da coluna IMC
Q1 = df_dummies['IMC'].quantile(0.25)
Q3 = df_dummies['IMC'].quantile(0.75)
IQR = Q3-Q1
limite_inferior = Q1 - 1.5*IQR
limite_superior = Q3 + 1.5*IQR

outliers_index = (df_dummies['IMC'] < limite_inferior) | (df_dummies['IMC'] > limite_superior)
df_sem_outliers = df_dummies[~outliers_index]
df_sem_outliers.reset_index(drop = True, inplace = True)

Explorando a correlação das variável Sono Tranquilo e Horas de Sono para o conjunto de dados

In [6]:
# Definindo a variável dependente (y) e a variável independente (X)
X = df_sem_outliers[['Sono_tranquilo']]  # Variável independente
X = sm.add_constant(X)  # Adiciona uma constante para o intercepto
y = df_sem_outliers['Diabetes']      # Variável dependente

# Ajustando o modelo de regressão logística
model = sm.Logit(y, X)
result = model.fit()

# Exibindo um resumo dos resultados
print(result.summary())

# Interpretando os coeficientes
coef = result.params['Sono_tranquilo']
p_value = result.pvalues['Sono_tranquilo']

alpha = 0.05
print(f'\nCoeficiente para Sono_tranquilo: {coef}')
print(f'Valor p: {p_value}')

if p_value < alpha:
    print('Rejeitamos a hipótese nula - Existe uma associação significativa entre as horas de sono tranquilo e a presença de diabetes.')
else:
    print('Não rejeitamos a hipótese nula - Não há evidências suficientes de uma associação significativa entre as horas de sono tranquiloz e a presença de diabetes.')


# Definindo a variável dependente (y) e a variável independente (X)
X = df_sem_outliers[['Horas_sono']]  # Variável independente
X = sm.add_constant(X)  # Adiciona uma constante para o intercepto
y = df_sem_outliers['Diabetes']      # Variável dependente

# Ajustando o modelo de regressão logística
model = sm.Logit(y, X)
result = model.fit()

# Exibindo um resumo dos resultados
print(result.summary())

# Interpretando os coeficientes
coef = result.params['Horas_sono']
p_value = result.pvalues['Horas_sono']

alpha = 0.05
print(f'\nCoeficiente para Horas_sono: {coef}')
print(f'Valor p: {p_value}')

if p_value < alpha:
    print('Rejeitamos a hipótese nula - Existe uma associação significativa entre as horas de sono  e a presença de diabetes.')
else:
    print('Não rejeitamos a hipótese nula - Não há evidências suficientes de uma associação significativa entre as horas de sono  e a presença de diabetes.')

#Criando uma cópia do DataFrame que será utilizado para o modelo de Machine Learning
df_limpo = df_sem_outliers.drop(['Horas_sono','Sono_tranquilo'], axis = 1).copy()



Optimization terminated successfully.
         Current function value: 0.613968
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:               Diabetes   No. Observations:                  296
Model:                          Logit   Df Residuals:                      294
Method:                           MLE   Df Model:                            1
Date:                Wed, 27 Nov 2024   Pseudo R-squ.:               0.0004759
Time:                        16:31:28   Log-Likelihood:                -181.73
converged:                       True   LL-Null:                       -181.82
Covariance Type:            nonrobust   LLR p-value:                    0.6774
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
const             -0.9833      0.395     -2.487      0.013      -1.758      -0.208
Sono_tranquilo   

Gerando modelos de Machine Learning

In [7]:
#Divisão entre treino e teste
x = df_limpo.drop('Diabetes', axis = 1)
y = df_limpo['Diabetes']

x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, stratify = y, random_state = 5, test_size = 0.25)

#Utilizando o DummyClassifier como modelo de Based Line
dummy = DummyClassifier()
dummy.fit(x_treino, y_treino)

#Árvore de Decisão
arvore = DecisionTreeClassifier(max_depth = 3, random_state = 5)
arvore.fit(x_treino, y_treino)

#KNN
normalizacao = MinMaxScaler() #normalizando os dados para utilizar o modelo
x_treino_normalizado = normalizacao.fit_transform(x_treino)
knn = KNeighborsClassifier()
knn.fit(x_treino_normalizado, y_treino)
x_teste_normalizado = normalizacao.transform(x_teste)

#SVC
SEED = 5
np.random.seed(SEED)
raw_treino_x, raw_teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25,
                                                         stratify = y)
#Ajustando a escala para o SVC
scaler = StandardScaler()
scaler.fit(raw_treino_x)
treino_x = scaler.transform(raw_treino_x)
teste_x = scaler.transform(raw_teste_x)

modelo = SVC(gamma='auto')
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

acuracia_svc = accuracy_score(teste_y, previsoes) * 100

#Acurácia dos modelos treinados
print(f'Acurácia Dummy: {dummy.score(x_teste, y_teste)}')
print(f'Acurácia Árvore: {arvore.score(x_teste, y_teste)}')
print(f'Acurácia KNN: {knn.score(x_teste_normalizado, y_teste)}')
print(f'Acurácia SVC: {acuracia_svc}')


Acurácia Dummy: 0.6891891891891891
Acurácia Árvore: 0.8243243243243243
Acurácia KNN: 0.7702702702702703
Acurácia SVC: 86.48648648648648
